In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev2\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost

In [2]:
import math
from datetime import datetime,timedelta
import numpy as np
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR 
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import KNeighborsClassifier

In [3]:
log_base = np.e

In [4]:
df_final = pandas.read_csv('phase2_trainingset.csv')

In [85]:
def delete_abnormal(df, tg, dire, time):
    df_temp = df_final_sorted[(df_final_sorted['tollgate_id'] == tg) & (df_final_sorted['direction'] == dire) &\
                (df_final_sorted['time'] == time)]\
                 .reset_index().drop(['index'],axis=1)
    volume_temp = df_temp['volume']
    mean_temp = volume_temp.mean()
    length = len(df_temp)
    for i in xrange(length):
        if np.abs(volume_temp[i] - mean_temp) > 8 * mean_temp:
            df_temp = df_temp.drop(i)
    return df_temp.reset_index().drop('index',axis=1)

In [86]:
df_final_sorted = df_final.sort_values(['time','tollgate_id','direction']).reset_index().drop(['index'],axis=1)
df_final_normal = pandas.DataFrame({})
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 1, 0, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 1, 1, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 2, 0, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 3, 0, i), ignore_index=True)
for i in xrange(72):
    df_final_normal = df_final_normal.append(delete_abnormal(df_final_sorted, 3, 1, i), ignore_index=True)
df_final_normal

,precipitation2,precipitation1,precipitation0,time71,time70,time69,time68,time67,time66,time65,...,type0lv2_log,type0lv3_log,type0lv4_log,type0lv5_log,model1lv0_log,model1lv1_log,model1lv2_log,model1lv3_log,model1lv4_log,model1lv5_log
0,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.397895,1.945910,1.945910,2.079442,2.397895,2.302585
1,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,1.609438,2.302585,1.386294,0.693147,1.791759,1.791759
2,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,3.295837,2.397895,2.639057,2.772589,1.945910,2.079442
3,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.484907,2.197225,1.791759,2.564949,2.302585,2.639057
4,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.079442,2.079442,2.079442,1.098612,1.945910,1.098612
5,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,3.091042,2.197225,2.484907,1.386294,0.693147,1.791759
6,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.833213,2.484907,2.639057,1.791759,2.079442,1.098612
7,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.772589,2.833213,2.397895,2.302585,2.564949,2.197225
8,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.197225,2.564949,1.386294,1.386294,1.609438,1.609438
9,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,2.944439,2.772589,2.708050,2.197225,0.693147,1.386294


In [87]:
df_final_normal['date_temp'] = df_final_normal['time_start'].str.split('-',expand=True)[2]\
                               .str.split(' ',expand=True)[0].apply(int)
df_final_sorted['date_temp'] = df_final_sorted['time_start'].str.split('-',expand=True)[2]\
                               .str.split(' ',expand=True)[0].apply(int)
#df_final_normal = df_final_normal[~((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] < 8))]

# six xgboost

In [8]:
df_remain10m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 28)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 30)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 11)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 19)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation10m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 30)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 11)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 19))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df10_appendm = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 30)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df_validation10am = df_validation10m.append(df10_appendm)

df_validation_final10m = df_validation10m.reset_index().drop(['index'],axis=1)
df_remain_final10m = df_remain10m.reset_index().drop(['index'],axis=1)
df_remain_shuffle10m = df_remain_final10m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle10m['volume_log'] = np.log(df_remain_shuffle10m['volume'])/np.log(log_base)
final_training_input10m = df_remain_shuffle10m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input10m = df_validation_final10m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]

In [9]:
df_remain11m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 10)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 11)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 13)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation11m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 10)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 11)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 13)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
# df11_appendm = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
# df_validation11am = df_validation11m.append(df11_appendm)

df_validation_final11m = df_validation11m.reset_index().drop(['index'],axis=1)
df_remain_final11m = df_remain11m.reset_index().drop(['index'],axis=1)
df_remain_shuffle11m = df_remain_final11m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle11m['volume_log'] = np.log(df_remain_shuffle11m['volume'])/np.log(log_base)
final_training_input11m = df_remain_shuffle11m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input11m = df_validation_final11m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]

In [109]:
df_remain20m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 21)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 6)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 20)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation20m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 21)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 6)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 20))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df20_append = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 12)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df_validation20ma = df_validation20m.append(df20_append)

df_validation_final20m = df_validation20m.reset_index().drop(['index'],axis=1)
df_remain_final20m = df_remain20m.reset_index().drop(['index'],axis=1)
df_remain_shuffle20m = df_remain_final20m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle20m['volume_log'] = np.log(df_remain_shuffle20m['volume'])/np.log(log_base)
final_training_input20m = df_remain_shuffle20m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input20m = df_validation_final20m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
final_validation_input20m

,tollgate_id2,tollgate_id1,tollgate_id0,direction,type0lv0_log,type0lv1_log,type0lv2_log,type0lv3_log,type0lv4_log,type0lv5_log,month,volumelv0_log,volumelv1_log,volumelv2_log,volumelv3_log,volumelv4_log,volumelv5_log,weekday,time,volume
0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9,1.791759,2.197225,2.639057,3.737670,3.610918,3.784190,2,24,37.0
1,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9,3.401197,3.433987,3.637586,4.276666,4.369448,4.605170,3,24,108.0
2,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.000000,0.000000,0.693147,1.098612,1.945910,2.564949,3,24,8.0
3,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10,3.367296,3.433987,3.806662,4.110874,4.595120,4.605170,4,24,125.0
4,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10,2.484907,3.526361,3.526361,3.688879,4.234107,4.060443,6,24,81.0
5,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10,3.178054,3.583519,3.828641,4.418841,4.465908,4.574711,1,24,120.0
6,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10,3.044522,3.135494,3.737670,4.330733,4.488636,4.718499,3,24,104.0
7,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9,1.791759,2.197225,2.639057,3.737670,3.610918,3.784190,2,25,30.0
8,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,9,3.401197,3.433987,3.637586,4.276666,4.369448,4.605170,3,25,110.0
9,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10,0.000000,0.000000,0.693147,1.098612,1.945910,2.564949,3,25,7.0


In [11]:
df_remain30m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 28)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 20)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation30m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 20))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
df30_appendm1 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)\
                             & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
# df30_appendm2 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
df_validation301m = df_validation30m.append(df30_appendm1)
# df_validation302m = df_validation301m.append(df30_appendm2)

df_validation_final30m = df_validation301m.reset_index().drop(['index'],axis=1)
df_remain_final30m = df_remain30m.reset_index().drop(['index'],axis=1)
df_remain_shuffle30m = df_remain_final30m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle30m['volume_log'] = np.log(df_remain_shuffle30m['volume'])/np.log(log_base)
final_training_input30m = df_remain_shuffle30m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input30m = df_validation_final30m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input30m

In [12]:
df_remain31m = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 29)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 29)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 21)))
                                & (df_final_normal['time'] < 30) & (df_final_normal['time'] > 23))]

df_validation31m = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 21))) \
                            & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df31_appendm1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)\
                             & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
# df31_appendm2 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)\
#                              & ((df_final_sorted['time'] < 30) & (df_final_sorted['time'] > 23))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df_validation311m = df_validation31m.append(df31_appendm1)
# df_validation312m = df_validation311m.append(df31_appendm2)

df_validation_final31m = df_validation311m.reset_index().drop(['index'],axis=1)
df_remain_final31m = df_remain31m.reset_index().drop(['index'],axis=1)
df_remain_shuffle31m = df_remain_final31m.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle31m['volume_log'] = np.log(df_remain_shuffle31m['volume'])/np.log(log_base)
final_training_input31m = df_remain_shuffle31m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input31m = df_validation_final31m.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input31m

In [13]:
input_dimension = 19
dataset = final_training_input10m.values
validation_set = final_validation_input10m.values
X_val_10m = validation_set[:, 0:input_dimension]
Y_val_10m= validation_set[:, input_dimension]
X_10m = dataset[:, 0:input_dimension]
Y_10m = dataset[:, input_dimension]
Y_log_10m = np.log(Y_val_10m)/np.log(log_base)
Y_log_10m

array([ 3.8918203 ,  4.85203026,  3.8286414 ,  3.4339872 ,  3.8918203 ,
        3.91202301,  3.87120101,  4.21950771,  4.91998093,  3.87120101,
        3.66356165,  3.8918203 ,  3.8286414 ,  3.91202301,  3.91202301,
        4.6443909 ,  3.8501476 ,  3.63758616,  3.68887945,  4.02535169,
        3.91202301,  3.71357207,  4.7095302 ,  4.09434456,  3.55534806,
        3.80666249,  3.76120012,  3.73766962,  3.95124372,  4.79579055,
        4.11087386,  3.4339872 ,  3.8501476 ,  3.98898405,  3.63758616,
        3.78418963,  4.91265489,  4.14313473,  3.63758616,  3.78418963,
        3.71357207,  4.07753744])

In [14]:
input_dimension = 19
dataset = final_training_input11m.values
validation_set = final_validation_input11m.values
X_val_11m = validation_set[:, 0:input_dimension]
Y_val_11m = validation_set[:, input_dimension]
X_11m = dataset[:, 0:input_dimension]
Y_11m = dataset[:, input_dimension]
Y_log_11m = np.log(Y_val_11m)/np.log(log_base)

In [110]:
input_dimension = 19
dataset = final_training_input20m.values
validation_set = final_validation_input20m.values
X_val_20m = validation_set[:, 0:input_dimension]
Y_val_20m = validation_set[:, input_dimension]
X_20m = dataset[:, 0:input_dimension]
Y_20m = dataset[:, input_dimension]
Y_log_20m = np.log(Y_val_20m)/np.log(log_base)
#Y_log_20

In [16]:
input_dimension = 19
dataset = final_training_input30m.values
validation_set = final_validation_input30m.values
X_val_30m = validation_set[:, 0:input_dimension]
Y_val_30m = validation_set[:, input_dimension]
X_30m = dataset[:, 0:input_dimension]
Y_30m = dataset[:, input_dimension]
Y_log_30m = np.log(Y_val_30m)/np.log(log_base)

In [17]:
input_dimension = 19
dataset = final_training_input31m.values
validation_set = final_validation_input31m.values
X_val_31m = validation_set[:, 0:input_dimension]
Y_val_31m = validation_set[:, input_dimension]
X_31m = dataset[:, 0:input_dimension]
Y_31m = dataset[:, input_dimension]
Y_log_31m = np.log(Y_val_31m)/np.log(log_base)

In [90]:
df_remain10a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 21)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 22)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 28)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 30)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 8)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 20)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation10a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 21)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 22)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 30)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 8)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 20))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df10_appenda = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 13)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 0)]
# df_validation10aa = df_validation10a.append(df10_appenda)

df_validation_final10a = df_validation10a.reset_index().drop(['index'],axis=1)
df_remain_final10a = df_remain10a.reset_index().drop(['index'],axis=1)
df_remain_shuffle10a = df_remain_final10a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle10a['volume_log'] = np.log(df_remain_shuffle10a['volume'])/np.log(log_base)
final_training_input10a = df_remain_shuffle10a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input10a = df_validation_final10a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
#final_validation_input10a

In [19]:
df_remain11a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 21)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 12)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 23)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation11a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 21)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 12)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 23))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
df11_appenda1 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)\
                             & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
df11_appenda2 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)\
                             & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 1) & (df_final_sorted['direction'] == 1)]
df_validation11aa1 = df_validation11a.append(df11_appenda1)
df_validation11aa2 = df_validation11aa1.append(df11_appenda2)

df_validation_final11a = df_validation11aa2.reset_index().drop(['index'],axis=1)
df_remain_final11a = df_remain11a.reset_index().drop(['index'],axis=1)
df_remain_shuffle11a = df_remain_final11a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle11a['volume_log'] = np.log(df_remain_shuffle11a['volume'])/np.log(log_base)
final_training_input11a = df_remain_shuffle11a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input11a = df_validation_final11a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
#final_validation_input11a

In [91]:
df_remain20a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 19)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 29)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 4)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 5)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 21)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation20a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 29)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 4)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 5)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 21))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df20_appenda1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df20_appenda2 = df_final_sorted[(df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 10)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 2) & (df_final_sorted['direction'] == 0)]
# df_validation20aa = df_validation20a.append(df20_appenda1)
# df_validation20aaa = df_validation20aa.append(df20_appenda2)

df_validation_final20a = df_validation20a.reset_index().drop(['index'],axis=1)
df_remain_final20a = df_remain20a.reset_index().drop(['index'],axis=1)
df_remain_shuffle20a = df_remain_final20a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle20a['volume_log'] = np.log(df_remain_shuffle20a['volume'])/np.log(log_base)
final_training_input20a = df_remain_shuffle20a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input20a = df_validation_final20a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input20a

In [21]:
df_remain30a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 20)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 23)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 14)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 17)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 19)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation30a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 20)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 23)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 14)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 17)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 19))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
# df30_appenda1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 0)]
# df_validation30aa = df_validation30a.append(df30_appenda1)

df_validation_final30a = df_validation30a.reset_index().drop(['index'],axis=1)
df_remain_final30a = df_remain30a.reset_index().drop(['index'],axis=1)
df_remain_shuffle30a = df_remain_final30a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle30a['volume_log'] = np.log(df_remain_shuffle30a['volume'])/np.log(log_base)
final_training_input30a = df_remain_shuffle30a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input30a = df_validation_final30a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input30a

In [22]:
df_remain31a = df_final_normal[~((((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 19)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 19)) | \
                                ((df_final_normal['month'] == 9) & (df_final_normal['date_temp'] == 25)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 9)) | \
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 11)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 16)) |\
                                ((df_final_normal['month'] == 10) & (df_final_normal['date_temp'] == 18)))
                                & (df_final_normal['time'] < 57) & (df_final_normal['time'] > 50))]

df_validation31a = df_final_sorted[(((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)) |\
                            ((df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 25)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 9)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 11)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 16)) |\
                            ((df_final_sorted['month'] == 10) & (df_final_sorted['date_temp'] == 18))) \
                            & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df31_appenda1 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 19)\
                             & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
                            & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
# df31_appenda2 = df_final_sorted[(df_final_sorted['month'] == 9) & (df_final_sorted['date_temp'] == 28)\
#                              & ((df_final_sorted['time'] < 57) & (df_final_sorted['time'] > 50))\
#                             & (df_final_sorted['tollgate_id'] == 3) & (df_final_sorted['direction'] == 1)]
df_validation31aa = df_validation31a.append(df31_appenda1)
# df_validation31aaa = df_validation31aa.append(df31_appenda2)

df_validation_final31a = df_validation31aa.reset_index().drop(['index'],axis=1)
df_remain_final31a = df_remain31a.reset_index().drop(['index'],axis=1)
df_remain_shuffle31a = df_remain_final31a.sample(frac=1,random_state=15).reset_index().drop(['index'],axis=1)
df_remain_shuffle31a['volume_log'] = np.log(df_remain_shuffle31a['volume'])/np.log(log_base)
final_training_input31a = df_remain_shuffle31a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                            ,'precipitation'
#                                                     ,'hour'
                                           ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                           ,'weekday','time','volume_log']]
final_validation_input31a = df_validation_final31a.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
# final_validation_input31a

In [92]:
input_dimension = 19
dataset = final_training_input10a.values
validation_set = final_validation_input10a.values
X_val_10a = validation_set[:, 0:input_dimension]
Y_val_10a = validation_set[:, input_dimension]
X_10a = dataset[:, 0:input_dimension]
Y_10a = dataset[:, input_dimension]
Y_log_10a = np.log(Y_val_10a)/np.log(log_base)
Y_log_10a

array([ 3.8501476 ,  3.36729583,  4.14313473,  3.4339872 ,  4.46590812,
        3.61091791,  3.63758616,  3.55534806,  3.58351894,  3.8918203 ,
        3.61091791,  4.58496748,  3.13549422,  3.80666249,  2.99573227,
        3.49650756,  3.73766962,  3.4339872 ,  4.89034913,  3.25809654,
        3.55534806,  2.99573227,  3.25809654,  4.02535169,  3.25809654,
        4.91998093,  2.99573227,  3.29583687,  2.99573227,  3.04452244,
        3.63758616,  3.17805383,  5.14749448,  3.33220451,  2.77258872,
        3.17805383,  3.17805383,  3.4339872 ,  2.89037176,  5.04985601,
        2.99573227,  2.83321334])

In [24]:
input_dimension = 19
dataset = final_training_input11a.values
validation_set = final_validation_input11a.values
X_val_11a = validation_set[:, 0:input_dimension]
Y_val_11a = validation_set[:, input_dimension]
X_11a = dataset[:, 0:input_dimension]
Y_11a = dataset[:, input_dimension]
Y_log_11a = np.log(Y_val_11a)/np.log(log_base)
Y_log_11a

array([ 4.54329478,  4.48863637,  4.58496748,  4.71849887,  4.72738782,
        4.69134788,  4.88280192,  4.8978398 ,  4.83628191,  4.65396035,
        4.54329478,  4.8598124 ,  4.7095302 ,  4.86753445,  4.66343909,
        4.30406509,  3.87120101,  4.33073334,  3.95124372,  4.34380542,
        4.44265126,  4.40671925,  4.34380542,  4.4308168 ,  4.51085951,
        4.35670883,  4.18965474,  4.02535169,  3.8918203 ,  3.8918203 ,
        4.71849887,  4.83628191,  4.86753445,  3.95124372,  4.4308168 ,
        3.8918203 ,  4.71849887,  4.83628191,  4.86753445,  3.95124372,
        4.4308168 ,  3.8918203 ])

In [93]:
input_dimension = 19
dataset = final_training_input20a.values
validation_set = final_validation_input20a.values
X_val_20a = validation_set[:, 0:input_dimension]
Y_val_20a = validation_set[:, input_dimension]
X_20a = dataset[:, 0:input_dimension]
Y_20a = dataset[:, input_dimension]
Y_log_20a = np.log(Y_val_20a)/np.log(log_base)
Y_log_20a

array([ 4.44265126,  4.55387689,  4.35670883,  2.94443898,  3.13549422,
        4.2341065 ,  4.49980967,  4.27666612,  4.52178858,  4.24849524,
        2.7080502 ,  2.77258872,  4.41884061,  4.31748811,  3.97029191,
        4.55387689,  4.26267988,  2.07944154,  2.48490665,  4.29045944,
        4.29045944,  4.00733319,  4.38202663,  4.24849524,  1.79175947,
        2.30258509,  4.24849524,  4.20469262,  3.91202301,  4.26267988,
        4.20469262,  2.19722458,  2.48490665,  3.87120101,  4.20469262,
        3.8918203 ,  4.38202663,  4.06044301,  1.94591015,  1.60943791,
        3.98898405,  3.98898405])

In [26]:
input_dimension = 19
dataset = final_training_input30a.values
validation_set = final_validation_input30a.values
X_val_30a = validation_set[:, 0:input_dimension]
Y_val_30a = validation_set[:, input_dimension]
X_30a = dataset[:, 0:input_dimension]
Y_30a = dataset[:, input_dimension]
Y_log_30a = np.log(Y_val_30a)/np.log(log_base)
Y_log_30a

array([ 4.83628191,  4.86753445,  4.98360662,  4.88280192,  4.94164242,
        4.69134788,  4.78749174,  4.75359019,  4.68213123,  4.80402104,
        4.75359019,  4.88280192,  4.58496748,  4.65396035,  4.55387689,
        4.91998093,  4.71849887,  4.78749174,  4.72738782,  4.47733681,
        4.56434819,  4.39444915,  4.58496748,  4.30406509,  4.65396035,
        4.49980967,  4.54329478,  4.36944785,  4.60517019,  4.8598124 ,
        4.21950771,  4.53259949,  4.53259949,  4.18965474,  4.21950771,
        4.14313473,  4.38202663,  4.04305127,  4.38202663,  4.17438727,
        3.93182563,  4.21950771])

In [27]:
input_dimension = 19
dataset = final_training_input31a.values
validation_set = final_validation_input31a.values
X_val_31a = validation_set[:, 0:input_dimension]
Y_val_31a = validation_set[:, input_dimension]
X_31a = dataset[:, 0:input_dimension]
Y_31a = dataset[:, input_dimension]
Y_log_31a = np.log(Y_val_31a)/np.log(log_base)
Y_log_31a

array([ 4.4543473 ,  4.77068462,  4.33073334,  4.58496748,  4.60517019,
        4.35670883,  4.44265126,  4.4308168 ,  4.51085951,  4.48863637,
        4.68213123,  4.31748811,  4.51085951,  4.40671925,  4.53259949,
        4.77068462,  4.60517019,  4.4543473 ,  4.29045944,  4.31748811,
        4.24849524,  4.35670883,  4.47733681,  4.29045944,  4.35670883,
        4.47733681,  4.38202663,  4.33073334,  4.69134788,  4.17438727,
        4.34380542,  4.26267988,  4.29045944,  4.31748811,  4.40671925,
        4.21950771,  4.4543473 ,  4.44265126,  4.51085951,  4.29045944,
        4.35670883,  4.34380542])

In [28]:
def mape(y,d):
    c=d.get_label()
    rec=np.sum(np.abs(np.power(log_base,y)-np.power(log_base,c))/np.power(log_base,c))/len(c)
    #rec=np.sum(np.abs(y-c)*1.0/c)/len(c)
    return "mape",rec

In [29]:
xlf_10a = xgboost.XGBRegressor(max_depth=12, 
                        learning_rate=0.005, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.9, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [30]:
xlf_10a.fit(X_10a,Y_10a, eval_metric=mape,verbose = True, eval_set = [(X_val_10a, Y_log_10a)],early_stopping_rounds=300)

[0]	validation_0-rmse:3.12678	validation_0-mape:0.946059
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.11032	validation_0-mape:0.945203
[2]	validation_0-rmse:3.09459	validation_0-mape:0.944377
[3]	validation_0-rmse:3.07954	validation_0-mape:0.94358
[4]	validation_0-rmse:3.06285	validation_0-mape:0.94267
[5]	validation_0-rmse:3.04772	validation_0-mape:0.941852
[6]	validation_0-rmse:3.03247	validation_0-mape:0.941008
[7]	validation_0-rmse:3.01739	validation_0-mape:0.940162
[8]	validation_0-rmse:3.00196	validation_0-mape:0.939284
[9]	validation_0-rmse:2.98575	validation_0-mape:0.938314
[10]	validation_0-rmse:2.97049	validation_0-mape:0.937423
[11]	validation_0-rmse:2.95514	validation_0-mape:0.936485
[12]	validation_0-rmse:2.93893	validation_0-mape:0.935456
[13]	validation_0-rmse:2.9243	validation_0-mape:0.934571
[14]	validation_0-rmse:2.90959	validation

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.005, max_delta_step=0, max_depth=12,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.9)

In [31]:
xlf_11a = xgboost.XGBRegressor(max_depth=10, 
                        learning_rate=0.06, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [32]:
xlf_11a.fit(X_11a,Y_11a, eval_metric=mape,verbose = True, eval_set = [(X_val_11a, Y_log_11a)],early_stopping_rounds=300)

[0]	validation_0-rmse:3.7383	validation_0-mape:0.97439
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.51592	validation_0-mape:0.967982
[2]	validation_0-rmse:3.31644	validation_0-mape:0.961063
[3]	validation_0-rmse:3.11898	validation_0-mape:0.952514
[4]	validation_0-rmse:2.9348	validation_0-mape:0.94285
[5]	validation_0-rmse:2.76058	validation_0-mape:0.931896
[6]	validation_0-rmse:2.61291	validation_0-mape:0.9215
[7]	validation_0-rmse:2.47309	validation_0-mape:0.91018
[8]	validation_0-rmse:2.32862	validation_0-mape:0.896105
[9]	validation_0-rmse:2.19245	validation_0-mape:0.88124
[10]	validation_0-rmse:2.06039	validation_0-mape:0.864308
[11]	validation_0-rmse:1.9492	validation_0-mape:0.848511
[12]	validation_0-rmse:1.8434	validation_0-mape:0.832087
[13]	validation_0-rmse:1.74545	validation_0-mape:0.815331
[14]	validation_0-rmse:1.646	validation_0-mape:0

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.06, max_delta_step=0, max_depth=10,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [94]:
xlf_20a = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.025, 
                        n_estimators=2000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [95]:
xlf_20a.fit(X_20a,Y_20a, eval_metric=mape,verbose = True, eval_set = [(X_val_20a, Y_log_20a)],early_stopping_rounds=300)

[0]	validation_0-rmse:3.24665	validation_0-mape:0.930739
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.1736	validation_0-mape:0.927058
[2]	validation_0-rmse:3.09841	validation_0-mape:0.923062
[3]	validation_0-rmse:3.02675	validation_0-mape:0.918986
[4]	validation_0-rmse:2.95532	validation_0-mape:0.91493
[5]	validation_0-rmse:2.88454	validation_0-mape:0.910611
[6]	validation_0-rmse:2.81452	validation_0-mape:0.905827
[7]	validation_0-rmse:2.74702	validation_0-mape:0.90085
[8]	validation_0-rmse:2.68155	validation_0-mape:0.89621
[9]	validation_0-rmse:2.6166	validation_0-mape:0.891025
[10]	validation_0-rmse:2.5537	validation_0-mape:0.886283
[11]	validation_0-rmse:2.49281	validation_0-mape:0.880899
[12]	validation_0-rmse:2.43213	validation_0-mape:0.874757
[13]	validation_0-rmse:2.37576	validation_0-mape:0.869955
[14]	validation_0-rmse:2.32057	validation_0-

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.025, max_delta_step=0, max_depth=8,
       min_child_weight=8, missing=None, n_estimators=2000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [35]:
xlf_30a = xgboost.XGBRegressor(max_depth=10, 
                        learning_rate=0.01, 
                        n_estimators=2000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [36]:
xlf_30a.fit(X_30a,Y_30a, eval_metric=mape,verbose = True, eval_set = [(X_val_30a, Y_log_30a)],early_stopping_rounds=300)

[0]	validation_0-rmse:4.03145	validation_0-mape:0.981445
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.9915	validation_0-mape:0.980698
[2]	validation_0-rmse:3.95171	validation_0-mape:0.979927
[3]	validation_0-rmse:3.91284	validation_0-mape:0.979142
[4]	validation_0-rmse:3.87405	validation_0-mape:0.978327
[5]	validation_0-rmse:3.83623	validation_0-mape:0.977492
[6]	validation_0-rmse:3.79814	validation_0-mape:0.976629
[7]	validation_0-rmse:3.76019	validation_0-mape:0.975736
[8]	validation_0-rmse:3.72361	validation_0-mape:0.974848
[9]	validation_0-rmse:3.68698	validation_0-mape:0.973924
[10]	validation_0-rmse:3.65056	validation_0-mape:0.972972
[11]	validation_0-rmse:3.61476	validation_0-mape:0.971992
[12]	validation_0-rmse:3.5795	validation_0-mape:0.970987
[13]	validation_0-rmse:3.54433	validation_0-mape:0.969962
[14]	validation_0-rmse:3.50913	validatio

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=10,
       min_child_weight=8, missing=None, n_estimators=2000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [37]:
xlf_31a = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.05, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [38]:
xlf_31a.fit(X_31a,Y_31a, eval_metric=mape,verbose = True, eval_set = [(X_val_31a, Y_log_31a)],early_stopping_rounds=300)

[0]	validation_0-rmse:3.73041	validation_0-mape:0.975721
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.54252	validation_0-mape:0.970698
[2]	validation_0-rmse:3.37247	validation_0-mape:0.965291
[3]	validation_0-rmse:3.20221	validation_0-mape:0.958842
[4]	validation_0-rmse:3.04822	validation_0-mape:0.952005
[5]	validation_0-rmse:2.89502	validation_0-mape:0.944051
[6]	validation_0-rmse:2.75744	validation_0-mape:0.935825
[7]	validation_0-rmse:2.63307	validation_0-mape:0.927389
[8]	validation_0-rmse:2.50081	validation_0-mape:0.917109
[9]	validation_0-rmse:2.37852	validation_0-mape:0.906306
[10]	validation_0-rmse:2.2585	validation_0-mape:0.89434
[11]	validation_0-rmse:2.15604	validation_0-mape:0.882986
[12]	validation_0-rmse:2.05303	validation_0-mape:0.870219
[13]	validation_0-rmse:1.95126	validation_0-mape:0.856288
[14]	validation_0-rmse:1.84995	validatio

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=8,
       min_child_weight=8, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [39]:
xlf_10m = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.01, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.9, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [40]:
xlf_10m.fit(X_10m,Y_10m, eval_metric=mape,verbose = True, eval_set = [(X_val_10m, Y_log_10m)],early_stopping_rounds=300)

[0]	validation_0-rmse:3.46032	validation_0-mape:0.965936
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.42508	validation_0-mape:0.964733
[2]	validation_0-rmse:3.39178	validation_0-mape:0.963589
[3]	validation_0-rmse:3.35652	validation_0-mape:0.962311
[4]	validation_0-rmse:3.32267	validation_0-mape:0.961029
[5]	validation_0-rmse:3.29101	validation_0-mape:0.959762
[6]	validation_0-rmse:3.25876	validation_0-mape:0.958501
[7]	validation_0-rmse:3.22621	validation_0-mape:0.957162
[8]	validation_0-rmse:3.1927	validation_0-mape:0.955733
[9]	validation_0-rmse:3.1611	validation_0-mape:0.954366
[10]	validation_0-rmse:3.12882	validation_0-mape:0.952888
[11]	validation_0-rmse:3.09823	validation_0-mape:0.951471
[12]	validation_0-rmse:3.06765	validation_0-mape:0.949987
[13]	validation_0-rmse:3.04021	validation_0-mape:0.948682
[14]	validation_0-rmse:3.00983	validatio

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.9)

In [41]:
xlf_11m = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.01, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [42]:
xlf_11m.fit(X_11m,Y_11m, eval_metric=mape,verbose = True, eval_set = [(X_val_11m, Y_log_11m)],early_stopping_rounds=300)

[0]	validation_0-rmse:4.16173	validation_0-mape:0.984271
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:4.12268	validation_0-mape:0.983644
[2]	validation_0-rmse:4.0841	validation_0-mape:0.983001
[3]	validation_0-rmse:4.04587	validation_0-mape:0.982338
[4]	validation_0-rmse:4.00801	validation_0-mape:0.981656
[5]	validation_0-rmse:3.97085	validation_0-mape:0.980961
[6]	validation_0-rmse:3.9337	validation_0-mape:0.98024
[7]	validation_0-rmse:3.89689	validation_0-mape:0.979499
[8]	validation_0-rmse:3.86046	validation_0-mape:0.978738
[9]	validation_0-rmse:3.82488	validation_0-mape:0.977967
[10]	validation_0-rmse:3.78913	validation_0-mape:0.977165
[11]	validation_0-rmse:3.75398	validation_0-mape:0.976348
[12]	validation_0-rmse:3.71897	validation_0-mape:0.975505
[13]	validation_0-rmse:3.68527	validation_0-mape:0.974667
[14]	validation_0-rmse:3.65133	validation

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=8,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [131]:
xlf_20m = xgboost.XGBRegressor(max_depth=12, 
                        learning_rate=0.06, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [132]:
xlf_20m.fit(X_20m,Y_20m, eval_metric=mape,verbose = True, eval_set = [(X_val_20m, Y_log_20m)],early_stopping_rounds=300)

[0]	validation_0-rmse:3.59269	validation_0-mape:0.955981
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:3.38264	validation_0-mape:0.948137
[2]	validation_0-rmse:3.18512	validation_0-mape:0.939617
[3]	validation_0-rmse:3.00319	validation_0-mape:0.930591
[4]	validation_0-rmse:2.83116	validation_0-mape:0.919709
[5]	validation_0-rmse:2.67273	validation_0-mape:0.908744
[6]	validation_0-rmse:2.52218	validation_0-mape:0.896383
[7]	validation_0-rmse:2.38494	validation_0-mape:0.885314
[8]	validation_0-rmse:2.24756	validation_0-mape:0.871278
[9]	validation_0-rmse:2.12443	validation_0-mape:0.859169
[10]	validation_0-rmse:2.00724	validation_0-mape:0.843299
[11]	validation_0-rmse:1.89587	validation_0-mape:0.826466
[12]	validation_0-rmse:1.79453	validation_0-mape:0.809727
[13]	validation_0-rmse:1.70065	validation_0-mape:0.792758
[14]	validation_0-rmse:1.60894	validat

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.06, max_delta_step=0, max_depth=12,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [45]:
xlf_30m = xgboost.XGBRegressor(max_depth=8, 
                        learning_rate=0.008, 
                        n_estimators=2000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=5, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=0, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [46]:
xlf_30m.fit(X_30m,Y_30m, eval_metric=mape,verbose = True, eval_set = [(X_val_30m, Y_log_30m)],early_stopping_rounds=200)

[0]	validation_0-rmse:4.3318	validation_0-mape:0.986503
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 200 rounds.
[1]	validation_0-rmse:4.29764	validation_0-mape:0.986038
[2]	validation_0-rmse:4.26399	validation_0-mape:0.985561
[3]	validation_0-rmse:4.23018	validation_0-mape:0.985072
[4]	validation_0-rmse:4.19634	validation_0-mape:0.98456
[5]	validation_0-rmse:4.16315	validation_0-mape:0.984042
[6]	validation_0-rmse:4.13054	validation_0-mape:0.983517
[7]	validation_0-rmse:4.09769	validation_0-mape:0.982971
[8]	validation_0-rmse:4.06468	validation_0-mape:0.982403
[9]	validation_0-rmse:4.03224	validation_0-mape:0.981826
[10]	validation_0-rmse:3.99938	validation_0-mape:0.98122
[11]	validation_0-rmse:3.96713	validation_0-mape:0.980606
[12]	validation_0-rmse:3.93531	validation_0-mape:0.979982
[13]	validation_0-rmse:3.90458	validation_0-mape:0.979367
[14]	validation_0-rmse:3.87352	validation

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.008, max_delta_step=0, max_depth=8,
       min_child_weight=5, missing=None, n_estimators=2000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [47]:
xlf_31m = xgboost.XGBRegressor(max_depth=6, 
                        learning_rate=0.01, 
                        n_estimators=3000, 
                        silent=True, 
                        objective="reg:linear", 
                
                        gamma=0,
                        min_child_weight=8, 
                        max_delta_step=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8, 
                        colsample_bylevel=1, 
                        reg_alpha=1, 
                        reg_lambda=20, 
                        scale_pos_weight=1, 
                        seed=10, 
                        missing=None)

In [48]:
xlf_31m.fit(X_31m,Y_31m, eval_metric=mape,verbose = True, eval_set = [(X_val_31m, Y_log_31m)],early_stopping_rounds=300)

[0]	validation_0-rmse:4.20796	validation_0-mape:0.98498
Multiple eval metrics have been passed: 'validation_0-mape' will be used for early stopping.

Will train until validation_0-mape hasn't improved in 300 rounds.
[1]	validation_0-rmse:4.17096	validation_0-mape:0.984413
[2]	validation_0-rmse:4.13434	validation_0-mape:0.983831
[3]	validation_0-rmse:4.09825	validation_0-mape:0.983235
[4]	validation_0-rmse:4.06228	validation_0-mape:0.98262
[5]	validation_0-rmse:4.02669	validation_0-mape:0.981988
[6]	validation_0-rmse:3.99151	validation_0-mape:0.981342
[7]	validation_0-rmse:3.95671	validation_0-mape:0.980678
[8]	validation_0-rmse:3.92227	validation_0-mape:0.979999
[9]	validation_0-rmse:3.88817	validation_0-mape:0.979302
[10]	validation_0-rmse:3.85528	validation_0-mape:0.97861
[11]	validation_0-rmse:3.81998	validation_0-mape:0.977841
[12]	validation_0-rmse:3.78538	validation_0-mape:0.97706
[13]	validation_0-rmse:3.75266	validation_0-mape:0.976294
[14]	validation_0-rmse:3.71877	validation_

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6,
       min_child_weight=8, missing=None, n_estimators=3000, nthread=-1,
       objective='reg:linear', reg_alpha=1, reg_lambda=20,
       scale_pos_weight=1, seed=10, silent=True, subsample=0.8)

In [133]:
predict_validation_10m = np.power(log_base,xlf_10m.predict(X_val_10m,ntree_limit=xlf_10m.best_iteration+1))
predict_validation_11m = np.power(log_base,xlf_11m.predict(X_val_11m,ntree_limit=xlf_11m.best_iteration+1))
predict_validation_20m = np.power(log_base,xlf_20m.predict(X_val_20m,ntree_limit=xlf_20m.best_iteration+1))
predict_validation_30m = np.power(log_base,xlf_30m.predict(X_val_30m,ntree_limit=xlf_30m.best_iteration+1))
predict_validation_31m = np.power(log_base,xlf_31m.predict(X_val_31m,ntree_limit=xlf_31m.best_iteration+1))

predict_validation_10a = np.power(log_base,xlf_10a.predict(X_val_10a,ntree_limit=xlf_10a.best_iteration+1))
predict_validation_11a = np.power(log_base,xlf_11a.predict(X_val_11a,ntree_limit=xlf_11a.best_iteration+1))
predict_validation_20a = np.power(log_base,xlf_20a.predict(X_val_20a,ntree_limit=xlf_20a.best_iteration+1))
predict_validation_30a = np.power(log_base,xlf_30a.predict(X_val_30a,ntree_limit=xlf_30a.best_iteration+1))
predict_validation_31a = np.power(log_base,xlf_31a.predict(X_val_31a,ntree_limit=xlf_31a.best_iteration+1))
predict_validation = np.array(list(predict_validation_10m)+list(predict_validation_11m)+list(predict_validation_20m)\
                     +list(predict_validation_30m)+list(predict_validation_31m)+ list(predict_validation_10a)\
                     +list(predict_validation_11a)+list(predict_validation_20a)\
                     +list(predict_validation_30a)+list(predict_validation_31a))
Y_val = np.array(list(Y_val_10m)+list(Y_val_11m)+list(Y_val_20m)+list(Y_val_30m)+list(Y_val_31m)\
                +list(Y_val_10a)+list(Y_val_11a)+list(Y_val_20a)+list(Y_val_30a)+list(Y_val_31a))
#predict_validation = xlf.predict(X_val, ntree_limit=model.best_iteration+1)
print mean_squared_error(Y_val, predict_validation)  
MAPE = (np.abs(Y_val - predict_validation)*1.0/Y_val).mean()
print MAPE

295.113196384
0.135979099043


# prediction

In [50]:
df_weather_test = pandas.read_csv("C:/Users/Tony/Jupyter_File/volume_prediction/dataSet_phase2/weather (table 7)_2.csv")

In [51]:
def extract_test_tg_dir(df_current, tg_id, direction):
    df_tg_dir = df_current[(df_current['tollgate_id']==tg_id) & (df_current['direction']==direction)].\
                reset_index().drop(['index'],axis=1)
    return df_tg_dir

In [52]:
df_test_last2h = pandas.read_csv('test2_20min_avg_volume_etc_cartype.csv')
df_test_time_temp = df_test_last2h['time_window'].str.split('[',expand=True)[1].str.split(',',expand=True)[0]
df_test_last2h['time_start'] = df_test_time_temp
df_test2h_1_0 = extract_test_tg_dir(df_test_last2h, 1, 0)
df_test2h_1_1 = extract_test_tg_dir(df_test_last2h, 1, 1)
df_test2h_2_0 = extract_test_tg_dir(df_test_last2h, 2, 0)
df_test2h_3_0 = extract_test_tg_dir(df_test_last2h, 3, 0)
df_test2h_3_1 = extract_test_tg_dir(df_test_last2h, 3, 1)
df_test2h_1_0

,tollgate_id,time_window,direction,volume,model0,model1,model2,model3,model4,model5,model6,model7,type0,type1,has_etc1,time_start
0,1,"[2016-10-25 06:00:00,2016-10-25 06:20:00)",0,10,0,4,6,0,0,0,0,0,0,0,0,2016-10-25 06:00:00
1,1,"[2016-10-25 06:20:00,2016-10-25 06:40:00)",0,17,0,13,4,0,0,0,0,0,0,0,3,2016-10-25 06:20:00
2,1,"[2016-10-25 06:40:00,2016-10-25 07:00:00)",0,30,0,20,6,3,1,0,0,0,0,0,8,2016-10-25 06:40:00
3,1,"[2016-10-25 07:00:00,2016-10-25 07:20:00)",0,25,0,21,3,0,1,0,0,0,0,0,6,2016-10-25 07:00:00
4,1,"[2016-10-25 07:20:00,2016-10-25 07:40:00)",0,38,0,34,4,0,0,0,0,0,0,0,14,2016-10-25 07:20:00
5,1,"[2016-10-25 07:40:00,2016-10-25 08:00:00)",0,44,0,42,1,1,0,0,0,0,0,0,14,2016-10-25 07:40:00
6,1,"[2016-10-25 15:00:00,2016-10-25 15:20:00)",0,52,0,47,5,0,0,0,0,0,0,0,18,2016-10-25 15:00:00
7,1,"[2016-10-25 15:20:00,2016-10-25 15:40:00)",0,44,0,43,1,0,0,0,0,0,0,0,11,2016-10-25 15:20:00
8,1,"[2016-10-25 15:40:00,2016-10-25 16:00:00)",0,46,0,38,8,0,0,0,0,0,0,0,13,2016-10-25 15:40:00
9,1,"[2016-10-25 16:00:00,2016-10-25 16:20:00)",0,50,0,49,1,0,0,0,0,0,0,0,13,2016-10-25 16:00:00


In [53]:
def add_last2h_test_dimension(df_tg_dir, key):
    lv_6_8 = [0]*6
    lv_15_17 = [0]*6
    lv_6_8[0] = []
    lv_6_8[1] = []
    lv_6_8[2] = []
    lv_6_8[3] = []
    lv_6_8[4] = []
    lv_6_8[5] = []
    lv_15_17[0] = []
    lv_15_17[1] = []
    lv_15_17[2] = []
    lv_15_17[3] = []
    lv_15_17[4] = []
    lv_15_17[5] = []
    for i in xrange(len(df_tg_dir)):
        if int(pandas.to_datetime(df_tg_dir['time_start'][i]).hour) < 12:
            lv_6_8[i%6].append(df_tg_dir[key][i])
        else:
            lv_15_17[i%6].append(df_tg_dir[key][i])
    return lv_6_8,lv_15_17

In [54]:
model_insert = {}
type_insert = {}

lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'volume')
lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'volume')
lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'volume')
lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'volume')
lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'volume')
lm = [0]*6
la = [0]*6
l_insert = [0]*6
for i in xrange(6):
    lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
for i in xrange(6):
    la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
for i in xrange(6):
    l_insert[i] = lm[i] + la[i]
    
for j in xrange(8):
    lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'model'+'%d'%j)
    lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'model'+'%d'%j)
    lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'model'+'%d'%j)
    lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'model'+'%d'%j)
    lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'model'+'%d'%j)
    lm = [0]*6
    la = [0]*6
    model_insert[j] = [0]*6
    for i in xrange(6):
        lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
    for i in xrange(6):
        la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
    for i in xrange(6):
        model_insert[j][i] = lm[i] + la[i]
    
for j in xrange(2):
    lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'type'+'%d'%j)
    lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'type'+'%d'%j)
    lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'type'+'%d'%j)
    lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'type'+'%d'%j)
    lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'type'+'%d'%j)
    lm = [0]*6
    la = [0]*6
    type_insert[j] = [0]*6
    for i in xrange(6):
        lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
    for i in xrange(6):
        la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
    for i in xrange(6):
        type_insert[j][i] = lm[i] + la[i]

lm_10,la_10=add_last2h_test_dimension(df_test2h_1_0, 'has_etc1')
lm_11,la_11=add_last2h_test_dimension(df_test2h_1_1, 'has_etc1')
lm_20,la_20=add_last2h_test_dimension(df_test2h_2_0, 'has_etc1')
lm_30,la_30=add_last2h_test_dimension(df_test2h_3_0, 'has_etc1')
lm_31,la_31=add_last2h_test_dimension(df_test2h_3_1, 'has_etc1')
lm = [0]*6
la = [0]*6
has_etc1_insert = [0]*6
for i in xrange(6):
    lm[i] = lm_10[i]*6 + lm_11[i]*6 + lm_20[i]*6 + lm_30[i]*6 + lm_31[i]*6
for i in xrange(6):
    la[i] = la_10[i]*6 + la_11[i]*6 + la_20[i]*6 + la_30[i]*6 + la_31[i]*6
for i in xrange(6):
    has_etc1_insert[i] = lm[i] + la[i]

In [55]:
df_test = pandas.read_csv('phase2_submission_sample_volume.csv')
df_test_temp = df_test['time_window'].str.split('[',expand=True)[1].str.split(',',expand=True)[0]
df_test['time_start'] = df_test_temp
df_test['volume'+'lv0'] = l_insert[0]
df_test['volume'+'lv1'] = l_insert[1]
df_test['volume'+'lv2'] = l_insert[2]
df_test['volume'+'lv3'] = l_insert[3]
df_test['volume'+'lv4'] = l_insert[4]
df_test['volume'+'lv5'] = l_insert[5]
for i in xrange(8):
    df_test['model'+('%d'%i)+'lv0'] = model_insert[i][0]
    df_test['model'+('%d'%i)+'lv1'] = model_insert[i][1]
    df_test['model'+('%d'%i)+'lv2'] = model_insert[i][2]
    df_test['model'+('%d'%i)+'lv3'] = model_insert[i][3]
    df_test['model'+('%d'%i)+'lv4'] = model_insert[i][4]
    df_test['model'+('%d'%i)+'lv5'] = model_insert[i][5]
for i in xrange(2):
    df_test['type'+('%d'%i)+'lv0'] = type_insert[i][0]
    df_test['type'+('%d'%i)+'lv1'] = type_insert[i][1]
    df_test['type'+('%d'%i)+'lv2'] = type_insert[i][2]
    df_test['type'+('%d'%i)+'lv3'] = type_insert[i][3]
    df_test['type'+('%d'%i)+'lv4'] = type_insert[i][4]
    df_test['type'+('%d'%i)+'lv5'] = type_insert[i][5]
df_test['has_etc1'+'lv0'] = has_etc1_insert[0]
df_test['has_etc1'+'lv1'] = has_etc1_insert[1]
df_test['has_etc1'+'lv2'] = has_etc1_insert[2]
df_test['has_etc1'+'lv3'] = has_etc1_insert[3]
df_test['has_etc1'+'lv4'] = has_etc1_insert[4]
df_test['has_etc1'+'lv5'] = has_etc1_insert[5]
df_test_new = df_test.drop(['time_window'],axis=1)
df_test_new

,tollgate_id,direction,volume,time_start,volumelv0,volumelv1,volumelv2,volumelv3,volumelv4,volumelv5,...,type1lv2,type1lv3,type1lv4,type1lv5,has_etc1lv0,has_etc1lv1,has_etc1lv2,has_etc1lv3,has_etc1lv4,has_etc1lv5
0,1,0,57.06,2016-10-25 08:00:00,10,17,30,25,38,44,...,0,0,0,0,0,3,8,6,14,14
1,1,0,261.65,2016-10-26 08:00:00,12,9,25,31,32,45,...,0,0,0,0,3,1,6,7,10,13
2,1,0,161.31,2016-10-27 08:00:00,14,14,23,20,36,42,...,0,0,0,0,1,3,8,7,13,11
3,1,0,266.66,2016-10-28 08:00:00,8,21,14,28,37,46,...,0,0,0,0,0,6,5,6,12,16
4,1,0,183.69,2016-10-29 08:00:00,11,17,19,27,37,24,...,0,0,0,0,4,5,5,4,5,7
5,1,0,198.50,2016-10-30 08:00:00,18,25,30,49,40,74,...,0,0,0,0,1,2,5,6,4,10
6,1,0,189.78,2016-10-31 08:00:00,27,54,60,95,98,106,...,0,0,0,0,2,5,8,8,16,15
7,1,0,67.83,2016-10-25 08:20:00,10,17,30,25,38,44,...,0,0,0,0,0,3,8,6,14,14
8,1,0,190.16,2016-10-26 08:20:00,12,9,25,31,32,45,...,0,0,0,0,3,1,6,7,10,13
9,1,0,115.11,2016-10-27 08:20:00,14,14,23,20,36,42,...,0,0,0,0,1,3,8,7,13,11


In [56]:
def holiday_judge_out(df_current, df_timewindow):
    judge_list=[]
    for i in xrange(len(df_current)): 
        start_time = datetime.strptime(df_timewindow[i], "%Y-%m-%d %H:%M:%S")
        if (start_time.month == 10) and ((start_time.day/4) == 0) and (df_current['direction'][i] == 0):
            judge_list.append(1)
        else:
            judge_list.append(0)
    df_current['holiday_ornot_out'] = judge_list
    return df_current

In [57]:
def holiday_judge_in(df_current, df_timewindow):
    judge_list=[]
    for i in xrange(len(df_current)): 
        start_time = datetime.strptime(df_timewindow[i], "%Y-%m-%d %H:%M:%S")
        if (start_time.month == 10) and ((start_time.day == 4) or (start_time.day == 5) or (start_time.day == 6)\
                                    or (start_time.day == 7)) and (df_current['direction'][i] == 1):
            judge_list.append(1)
        else:
            judge_list.append(0)
    df_current['holiday_ornot_in'] = judge_list
    return df_current

In [58]:
def insert_weather(df_current, df_timewindow, df_weather):
    pre_list = []
    wind_list = []
    tempra_list = []
    humidity_list = []
    a = df_timewindow.str.split(' ',expand=True)[0]
    b = df_timewindow.str.split(' ',expand=True)[1]
    c = b.str.split(':',expand=True)[0]
    for i in xrange(len(df_current)):
        date = a[i]
        hour = c[i]
        temp0 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['precipitation']
        temp1 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['wind_speed']
        temp2 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['temperature']
        temp3 = df_weather[(df_weather['date']==date) & (df_weather['hour']==int(hour))]['rel_humidity']
        '''
        if len(temp0) == 0:
            temp0 = 0
        if len(temp1) == 0:
            temp1 = 0
        if len(temp2) == 0:
            temp2 = 0
        if len(temp3) == 0:
            temp3 = 0
        '''
        if ((float(temp0) > 0) and (float(temp0) < 2.5)):
            temp0 = 1
        elif float(temp0) >= 2.5:
            temp0 = 2
        if (float(temp1) <= 0.4):
            temp1 = 0
        elif ((float(temp1) > 0.4) and (float(temp1) <= 1.5)):
            temp1 = 1
        elif ((float(temp1) > 1.5) and (float(temp1) <= 5)):
            temp1 = 2
        elif (float(temp1) > 5):
            temp1 =3
        pre_list.append(float(temp0)) 
        wind_list.append(float(temp1))
        tempra_list.append(float(temp2))
        humidity_list.append(float(temp3))
    df_current['precipitation'] = pre_list
    df_current['wind_speed'] = wind_list
    df_current['temperature'] = tempra_list
    df_current['humidity'] = humidity_list
    return df_current

In [59]:
def transfer_volume(df_temp2):
    df_transfer = df_temp2.drop(['volume'],axis=1)
    df_transfer['volume'] = df_temp2['volume']
    return df_transfer

In [60]:
def add_time_dimension(df_input, df_timewindow):
    m_list=[]
    weekday_list=[]
    time_list=[]
    hour_list = []
    for i in xrange(len(df_input)):
        start_time = datetime.strptime(df_timewindow[i], "%Y-%m-%d %H:%M:%S")
        m_list.append(start_time.month)
        hour_list.append(start_time.hour)
        if str(start_time.date()) == '2016-10-08':
            weekday_list.append(3)
        elif str(start_time.date()) == '2016-10-09':
            weekday_list.append(4)
        else:
            weekday_list.append(start_time.weekday())
        time_list.append(start_time.hour*3 + (start_time.minute/20))
    df_input['month'] = m_list
    df_input['weekday'] = weekday_list
    df_input['time'] = time_list
    df_input['hour'] = hour_list
    return df_input

In [61]:
df_time_temp_test = df_test_new['time_start']
# df_insert_weather_test = insert_weather(df_test_new, df_time_temp_test, df_weather_test)
df_add_time_test = add_time_dimension(df_test_new, df_time_temp_test)
df_judge_out = holiday_judge_out(df_add_time_test, df_time_temp_test)
df_h_judge_test = holiday_judge_in(df_judge_out, df_time_temp_test)
df_final_test = transfer_volume(df_h_judge_test)
df_final_test

,tollgate_id,direction,time_start,volumelv0,volumelv1,volumelv2,volumelv3,volumelv4,volumelv5,model0lv0,...,has_etc1lv3,has_etc1lv4,has_etc1lv5,month,weekday,time,hour,holiday_ornot_out,holiday_ornot_in,volume
0,1,0,2016-10-25 08:00:00,10,17,30,25,38,44,0,...,6,14,14,10,1,24,8,0,0,57.06
1,1,0,2016-10-26 08:00:00,12,9,25,31,32,45,0,...,7,10,13,10,2,24,8,0,0,261.65
2,1,0,2016-10-27 08:00:00,14,14,23,20,36,42,0,...,7,13,11,10,3,24,8,0,0,161.31
3,1,0,2016-10-28 08:00:00,8,21,14,28,37,46,0,...,6,12,16,10,4,24,8,0,0,266.66
4,1,0,2016-10-29 08:00:00,11,17,19,27,37,24,0,...,4,5,7,10,5,24,8,0,0,183.69
5,1,0,2016-10-30 08:00:00,18,25,30,49,40,74,0,...,6,4,10,10,6,24,8,0,0,198.50
6,1,0,2016-10-31 08:00:00,27,54,60,95,98,106,0,...,8,16,15,10,0,24,8,0,0,189.78
7,1,0,2016-10-25 08:20:00,10,17,30,25,38,44,0,...,6,14,14,10,1,25,8,0,0,67.83
8,1,0,2016-10-26 08:20:00,12,9,25,31,32,45,0,...,7,10,13,10,2,25,8,0,0,190.16
9,1,0,2016-10-27 08:20:00,14,14,23,20,36,42,0,...,7,13,11,10,3,25,8,0,0,115.11


In [62]:
df_final_test['time_hour'] = df_final_test['time_start'].str.split('-',expand=True)[2]\
                             .str.split(' ',expand=True)[1].str.split(':',expand=True)[0].apply(int)
df_final_test['date'] = df_final_test['time_start'].str.split('-',expand=True)[2]\
                             .str.split(' ',expand=True)[0].apply(int)
df_final_test['minute'] = df_final_test['time_start'].str.split('-',expand=True)[2]\
                             .str.split(' ',expand=True)[1].str.split(':',expand=True)[1].apply(int)

In [63]:
df_final_test['travel_time_B1'] = [0] * 420
df_final_test['travel_time_C1'] = [0] * 420
df_final_test['travel_time_A2'] = [0] * 420
df_final_test['travel_time_A3'] = [0] * 420
df_final_test['travel_time_B3'] = [0] * 420
df_final_test['travel_time_C3'] = [0] * 420

In [64]:
def insert_onehot(df, key):
    df_temp = pandas.get_dummies(df[key]) 
    length = df_temp.shape[1]
    df_temp.columns=[i for i in xrange(length)]
    for i in xrange(length):
        df.insert(0, key+'%d'%i, df_temp[i])
    return df

In [65]:
insert_onehot(df_final_test, 'tollgate_id')
insert_onehot(df_final_test, 'month')
insert_onehot(df_final_test, 'weekday')
insert_onehot(df_final_test, 'time')
# insert_onehot(df_final_test, 'precipitation')
df_final_test

,time11,time10,time9,time8,time7,time6,time5,time4,time3,time2,...,volume,time_hour,date,minute,travel_time_B1,travel_time_C1,travel_time_A2,travel_time_A3,travel_time_B3,travel_time_C3
0,0,0,0,0,0,0,0,0,0,0,...,57.06,8,25,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,261.65,8,26,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,161.31,8,27,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,266.66,8,28,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,183.69,8,29,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,198.50,8,30,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,189.78,8,31,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,67.83,8,25,20,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,190.16,8,26,20,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,115.11,8,27,20,0,0,0,0,0,0


In [66]:
df_final_test.to_csv('testset_final.csv')

In [67]:
df_final_test['volumelv0_log'] = np.log(df_final_test['volumelv0'])/np.log(log_base)
df_final_test['volumelv1_log'] = np.log(df_final_test['volumelv1'])/np.log(log_base)
df_final_test['volumelv2_log'] = np.log(df_final_test['volumelv2'])/np.log(log_base)
df_final_test['volumelv3_log'] = np.log(df_final_test['volumelv3'])/np.log(log_base)
df_final_test['volumelv4_log'] = np.log(df_final_test['volumelv4'])/np.log(log_base)
df_final_test['volumelv5_log'] = np.log(df_final_test['volumelv5'])/np.log(log_base)
df_final_test['has_etc1lv0_log'] = np.log(df_final_test['has_etc1lv0']+1)/np.log(log_base)
df_final_test['has_etc1lv1_log'] = np.log(df_final_test['has_etc1lv1']+1)/np.log(log_base)
df_final_test['has_etc1lv2_log'] = np.log(df_final_test['has_etc1lv2']+1)/np.log(log_base)
df_final_test['has_etc1lv3_log'] = np.log(df_final_test['has_etc1lv3']+1)/np.log(log_base)
df_final_test['has_etc1lv4_log'] = np.log(df_final_test['has_etc1lv4']+1)/np.log(log_base)
df_final_test['has_etc1lv5_log'] = np.log(df_final_test['has_etc1lv5']+1)/np.log(log_base)
df_final_test['type0lv0_log'] = np.log(df_final_test['type0lv0']+1)/np.log(log_base)
df_final_test['type0lv1_log'] = np.log(df_final_test['type0lv1']+1)/np.log(log_base)
df_final_test['type0lv2_log'] = np.log(df_final_test['type0lv2']+1)/np.log(log_base)
df_final_test['type0lv3_log'] = np.log(df_final_test['type0lv3']+1)/np.log(log_base)
df_final_test['type0lv4_log'] = np.log(df_final_test['type0lv4']+1)/np.log(log_base)
df_final_test['type0lv5_log'] = np.log(df_final_test['type0lv5']+1)/np.log(log_base)
df_final_test['model1lv0_log'] = np.log(df_final_test['model1lv0']+1)/np.log(log_base)
df_final_test['model1lv1_log'] = np.log(df_final_test['model1lv1']+1)/np.log(log_base)
df_final_test['model1lv2_log'] = np.log(df_final_test['model1lv2']+1)/np.log(log_base)
df_final_test['model1lv3_log'] = np.log(df_final_test['model1lv3']+1)/np.log(log_base)
df_final_test['model1lv4_log'] = np.log(df_final_test['model1lv4']+1)/np.log(log_base)
df_final_test['model1lv5_log'] = np.log(df_final_test['model1lv5']+1)/np.log(log_base)

In [68]:
df_test_input_final = df_final_test.ix[:,['tollgate_id2','tollgate_id1','tollgate_id0','direction'\
#                                             ,'precipitation'
#                                                      ,'hour'
                                            ,'type0lv0_log','type0lv1_log','type0lv2_log'
                                           ,'type0lv3_log','type0lv4_log','type0lv5_log'
                                           ,'month'\
                                           ,'volumelv0_log','volumelv1_log','volumelv2_log'
                                           ,'volumelv3_log','volumelv4_log','volumelv5_log'  
#                                            ,'has_etc1lv0_log'
#                                            ,'has_etc1lv1_log'
#                                            ,'has_etc1lv2_log'
#                                            ,'has_etc1lv3_log'
#                                            ,'has_etc1lv4_log'
#                                            ,'has_etc1lv5_log'
                                            ,'weekday','time','volume']]
df_test_input_final

,tollgate_id2,tollgate_id1,tollgate_id0,direction,type0lv0_log,type0lv1_log,type0lv2_log,type0lv3_log,type0lv4_log,type0lv5_log,month,volumelv0_log,volumelv1_log,volumelv2_log,volumelv3_log,volumelv4_log,volumelv5_log,weekday,time,volume
0,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.302585,2.833213,3.401197,3.218876,3.637586,3.784190,1,24,57.06
1,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.484907,2.197225,3.218876,3.433987,3.465736,3.806662,2,24,261.65
2,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.639057,2.639057,3.135494,2.995732,3.583519,3.737670,3,24,161.31
3,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.079442,3.044522,2.639057,3.332205,3.610918,3.828641,4,24,266.66
4,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.397895,2.833213,2.944439,3.295837,3.610918,3.178054,5,24,183.69
5,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.890372,3.218876,3.401197,3.891820,3.688879,4.304065,6,24,198.50
6,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,3.295837,3.988984,4.094345,4.553877,4.584967,4.663439,0,24,189.78
7,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.302585,2.833213,3.401197,3.218876,3.637586,3.784190,1,25,67.83
8,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.484907,2.197225,3.218876,3.433987,3.465736,3.806662,2,25,190.16
9,0,0,1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,2.639057,2.639057,3.135494,2.995732,3.583519,3.737670,3,25,115.11


In [69]:
dataset_test = df_test_input_final.values
X_test = dataset_test[:, 0:input_dimension]
X_test

array([[  0.        ,   0.        ,   1.        , ...,   3.78418963,
          1.        ,  24.        ],
       [  0.        ,   0.        ,   1.        , ...,   3.80666249,
          2.        ,  24.        ],
       [  0.        ,   0.        ,   1.        , ...,   3.73766962,
          3.        ,  24.        ],
       ..., 
       [  1.        ,   0.        ,   0.        , ...,   4.35670883,
          5.        ,  56.        ],
       [  1.        ,   0.        ,   0.        , ...,   4.65396035,
          6.        ,  56.        ],
       [  1.        ,   0.        ,   0.        , ...,   4.21950771,
          0.        ,  56.        ]])

In [70]:
volume_predict_out1 = np.power(log_base, xlf_10m.predict(X_test[:42],ntree_limit=xlf_10m.best_iteration+1))

In [71]:
volume_predict_out2 = np.power(log_base, xlf_11m.predict(X_test[42:84],ntree_limit=xlf_11m.best_iteration+1))

In [134]:
volume_predict_out3 = np.power(log_base, xlf_20m.predict(X_test[84:126],ntree_limit=xlf_20m.best_iteration+1))

In [73]:
volume_predict_out4 = np.power(log_base, xlf_30m.predict(X_test[126:168],ntree_limit=xlf_30m.best_iteration+1))

In [74]:
volume_predict_out5 = np.power(log_base, xlf_31m.predict(X_test[168:210],ntree_limit=xlf_31m.best_iteration+1))

In [75]:
volume_predict_out6 = np.power(log_base, xlf_10a.predict(X_test[210:252],ntree_limit=xlf_10a.best_iteration+1))

In [76]:
volume_predict_out7 = np.power(log_base, xlf_11a.predict(X_test[252:294],ntree_limit=xlf_11a.best_iteration+1))

In [135]:
volume_predict_out8 = np.power(log_base, xlf_20a.predict(X_test[294:336],ntree_limit=xlf_20a.best_iteration+1))

In [78]:
volume_predict_out9 = np.power(log_base, xlf_30a.predict(X_test[336:378],ntree_limit=xlf_30a.best_iteration+1))

In [79]:
volume_predict_out10 = np.power(log_base, xlf_31a.predict(X_test[378:420],ntree_limit=xlf_31a.best_iteration+1))

In [136]:
volume_predict_out = np.array(list(volume_predict_out1)+list(volume_predict_out2)+list(volume_predict_out3)+\
                             list(volume_predict_out4)+list(volume_predict_out5)+list(volume_predict_out6)+\
                             list(volume_predict_out7)+list(volume_predict_out8)+list(volume_predict_out9)+\
                             list(volume_predict_out10))
volume_predict_out

array([  46.62240219,   46.00836563,   42.9525795 ,   43.48701096,
         31.78225136,   64.18213654,  108.91495514,   47.86237335,
         48.1084938 ,   45.73730469,   46.09872818,   33.22319031,
         67.96261597,  110.03897095,   48.53292084,   48.75653839,
         46.25404739,   46.9257431 ,   33.69438553,   68.34642029,
        109.78797913,   48.89121628,   48.92844009,   46.62054825,
         47.78229904,   33.83887863,   69.29036713,  107.78026581,
         48.78919983,   48.83922195,   46.58203888,   47.78229904,
         33.83887863,   68.51724243,  102.85166168,   48.62804794,
         48.66866684,   46.41936493,   47.63048935,   33.67742538,
         66.61740875,   95.741745  ,  120.18707275,  115.80173492,
        114.40011597,  116.96006775,  103.90483093,   91.99016571,
        118.82038879,  120.69976807,  116.3315506 ,  115.01315308,
        117.90072632,  105.50888824,   93.76725769,  119.26638794,
        120.80854034,  116.44626617,  115.05363464,  118.01006

In [137]:
df_submission = pandas.read_csv('phase2_submission_sample_volume.csv')
df_submission_new = df_submission.drop(['volume'],axis=1)
df_submission_new['volume'] = volume_predict_out
df_submission_new.to_csv('phase2_submission_volume_06_01.csv',index=False)